In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam




In [ ]:
## Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()




170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
# Normalize the pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:

# One-hot encode the target labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)



In [ ]:
# Load the pre-trained VGG16 model without the classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))



58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Add a custom classification head on top of the VGG16 base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)



In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)



In [ ]:
# Freeze the pre-trained layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False



In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Data augmentation to increase dataset diversity
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(x_train)



In [ ]:
# Train the model
batch_size = 64
epochs = 10
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train) // batch_size,
          epochs=epochs, validation_data=(x_test, y_test))



781/781 [==============================] - 45s 46ms/step - loss: 1.5252 - accuracy: 0.4640 - val_loss: 1.3092 - val_accuracy: 0.5354
Epoch 2/10
781/781 [==============================] - 33s 42ms/step - loss: 1.3770 - accuracy: 0.5138 - val_loss: 1.2769 - val_accuracy: 0.5463
Epoch 3/10
781/781 [==============================] - 34s 43ms/step - loss: 1.3312 - accuracy: 0.5278 - val_loss: 1.2660 - val_accuracy: 0.5525
Epoch 4/10
781/781 [==============================] - 36s 46ms/step - loss: 1.3040 - accuracy: 0.5384 - val_loss: 1.2144 - val_accuracy: 0.5687
Epoch 5/10
781/781 [==============================] - 33s 43ms/step - loss: 1.2754 - accuracy: 0.5479 - val_loss: 1.1862 - val_accuracy: 0.5831
Epoch 6/10
781/781 [==============================] - 33s 42ms/step - loss: 1.2541 - accuracy: 0.5563 - val_loss: 1.2073 - val_accuracy: 0.5753
Epoch 7/10
781/781 [==============================] - 34s 43ms/step - loss: 1.2406 - accuracy: 0.5607 - val_loss: 1.1694 - val_accuracy: 0.5892
Epo

In [ ]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.1580147743225098
Test accuracy: 0.5989999771118164
